In [1]:
from fileoperations.fileoperations import getFileNamesInDir
from symbtrdataextractor import symbtrreader, extractor
import os
import json
import pandas as pd 

import pdb

In [2]:
symbtr_cols = ['Sira', 'Kod', 'Nota53', 'NotaAE', 'Koma53', 'KomaAE', 
               'Pay', 'Payda', 'Ms', 'LNS', 'Bas', 'Soz1', 'Offset']
symbtrfolder = os.path.join('..', '..', 'symbtr_corrections', 'usul')
savefolder = os.path.join('..', '..', 'symbtr_corrections', 'usul_name')

symbtrs = [os.path.splitext(s)[0] for s in getFileNamesInDir('../txt', keyword='*.txt')[2]]

usuldict = json.load(open('./unittests/data/usul_extended.json','r'))
mu2dict = {}
for key, val in usuldict.iteritems():
    for vrt in val['variants']:
        if vrt['mu2_name']:  # if it doesnt have a mu2 name, the usul is not in symbtr collection
            mu2dict[vrt['mu2_name']] = int(vrt['symbtr_internal_id'])
            
inv_mu2dict = {v: k for k, v in mu2dict.items()}

In [3]:
for ii, symbTr in enumerate(symbtrs):
    filechanged = False
    symbtrtxtfile = os.path.join(symbtrfolder, symbTr + '.txt')

    df = pd.read_csv(symbtrtxtfile, sep='\t', encoding='utf-8')
    for index, row in df.iterrows():     
        rowchanged = False
        # change null to empty string
        for key, val in row.iteritems():
            if pd.isnull(val):
                row[key] = ''
                rowchanged = True

        if row['Kod'] == 51:
            usul_id = row['LNS']
            usul_name = row['Soz1']
            
            if usul_name:  # name given
                # check if the usul pair matches with the mu2dict
                if mu2dict[usul_name] == usul_id:
                    pass
                else:
                    print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': ' + usul_name + ' and ' + str(usul_id) + ' does not match.'
            elif usul_id:
                print str(ii) + ' ' + symbTr + ', line ' + str(index) + ': Filling missing ' + inv_mu2dict[usul_id]
                row['Soz1'] = inv_mu2dict[usul_id]
                rowchanged = True
                filechanged = True
            else:
                pdb.set_trace()

        # reassign
        if rowchanged:
            df.iloc[index] = row
            
    if filechanged:
        savefile = os.path.join(savefolder, symbTr + '.txt')
        df.to_csv(savefile, sep='\t', index=False, encoding='utf-8')


7 muhayyerkurdi--sarki--nimsofyan--gonlumu_gonlune--omer_sami_gupgup, line 224: Filling missing Nîmsofyan
19 evic--sazsemaisi--aksaksemai----muallim_ismail_hakki_bey, line 736: Filling missing Aksaksemâî
26 mahur--sazsemaisi--aksaksemai--bahar_1--goksel_baktagir, line 546: Filling missing Semâî
26 mahur--sazsemaisi--aksaksemai--bahar_1--goksel_baktagir, line 755: Filling missing Aksaksemâî
73 guldeste--sazsemaisi--aksaksemai----dede_salih_efendi, line 374: Filling missing Aksaksemâî
108 tahirbuselik--sazsemaisi--aksaksemai----kemani_riza_efendi, line 529: Filling missing Aksaksemâî
129 yegah--sazsemaisi--aksaksemai----neyzen_aziz_dede, line 361: Filling missing Aksaksemâî
132 nihavent--sarki--semai--gelin_kizlar--haci_faik_bey, line 232: Filling missing Semâî
175 gulzar--sazsemaisi--aksaksemai----ismet_burkay, line 868: Filling missing Aksaksemâî
176 ussak--sazsemaisi--aksaksemai----dede_salih_efendi, line 708: Filling missing Aksaksemâî
179 evic--sazsemaisi--aksaksemai----sedat_oztopr

TypeError: coercing to Unicode: need string or buffer, numpy.int64 found

273 hicaz--sazsemaisi--aksaksemai----muhittin_erev, line 429: Semâî and 99 does not match.
